# Supervised Retrieval

In this notebook we use the supervised classification model for a supervised crosslingual information retrieval task.

## I. Import Data

In this section we import the feature dataframe for the retrieval task.

In [ ]:
import pandas as pdr
feature_retrieval = pd.read_json("../data/processed/feature_retrieval.json")

## II. Supervised Retrieval